# ENPMDA Usage

In [1]:
import tempfile
tempdir = tempfile.mkdtemp()
print(tempdir)

/tmp/tmpbeqxf9pi


In [2]:
import dask
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=32, scheduler_port=8786,
                      memory_limit='auto')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 32,Total memory: 125.78 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:8786,Workers: 32
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 125.78 GiB
Comm: tcp://127.0.0.1:46365,Total threads: 1
Dashboard: http://127.0.0.1:41979/status,Memory: 3.93 GiB
Nanny: tcp://127.0.0.1:37301,


In [3]:
from ENPMDA import MDDataFrame
from ENPMDA.preprocessing import TrajectoryEnsemble
from ENPMDATests.datafiles import ensemble_ala_tpr, ensemble_ala_traj, ensemble_ala_top

In [4]:
traj_ensembles = TrajectoryEnsemble(ensemble_name=tempdir + '/test_ensemble',
                                    topology_list=ensemble_ala_top,
                                    trajectory_list=ensemble_ala_traj,
                                    tpr_list=ensemble_ala_tpr,
                                    updating=True,
                                    only_raw=False)

dask finished
pickle raw_traj universe done
dask finished
pickle traj protein universe done
dask finished
pickle traj system universe done


![dask task stream](./preprocessing_traj.png)

In [5]:
md_dataframe = MDDataFrame(location=tempdir + '/test_dataframe')

In [6]:
md_dataframe.add_traj_ensemble(traj_ensembles, npartitions=16)

Dask dataframe generated with 16 partitions


In [7]:
from ENPMDA.analysis.analysis import *

In [8]:
class get_rmsd_init(DaskChunkMdanalysis):
    name = 'rmsd_to_init'

    def run_analysis(self, universe, start, stop, step):
        self._feature_info = ['rmsd_to_init']

        name_backbone = universe.select_atoms('backbone')
        rmsd = RMSD(name_backbone, name_backbone).run(start, stop, step)
        return rmsd.results['rmsd'].T[2]

In [9]:
# A sanity check will be performed to ensure that the analysis is working
# The parallel analysis will be actively done in the background.

md_dataframe.add_analysis(get_rmsd_init)
md_dataframe.add_analysis(get_backbonetorsion)
md_dataframe.add_analysis(get_atomic_position)
md_dataframe.add_analysis(get_protein_hydration)

![dask task stream](./analysis_parallel.png)

In [10]:
md_dataframe.save('test_analysis')

In [11]:
md_dataframe.dataframe

,universe,universe_system,system,md_name,frame,traj_time,stride,rmsd_to_init,torsion,at_position,protein_hydration
0,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,0,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,0,0.0,1,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...
1,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,0,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,1,2.0,1,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...
2,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,0,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,2,4.0,1,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...
3,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,0,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,3,6.0,1,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...
4,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,0,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,4,8.0,1,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...
...,...,...,...,...,...,...,...,...,...,...,...
163,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,7,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,16,32.0,1,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...
164,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,7,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,17,34.0,1,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...
165,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,7,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,18,36.0,1,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...
166,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,7,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,19,38.0,1,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...,/nethome/yzhuang/git_repo/ENPMDA/tests/ENPMDAT...
